# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/dm"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dm" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/itba2025-8d0a/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


ln: failed to create symbolic link '/content/buckets/b1/dm': File exists


limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656417,35.1,1454474,77.7,1321186,70.6
Vcells,1219191,9.4,8388608,64.0,1975128,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [3]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 100019
PARAM$qsemillas <- 1

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [4]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [5]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}


In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}


In [7]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2910"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [8]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [9]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [10]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 87

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [15]:
# ⚠️ OPCIONAL: Limpiar resultados anteriores si querés empezar de cero
tb_grid_search_detalle <- data.table()

# ⚠️ Eliminar archivo de resultados anterior si existe
if (file.exists("gridsearch_detalle.txt")) file.remove("gridsearch_detalle.txt")
if (file.exists("gridsearch_log.txt")) file.remove("gridsearch_log.txt")


# Verifica existencia de PARAM
if (!exists("PARAM")) stop("⚠️ El objeto global 'PARAM' no está definido. Defínelo antes de correr este script.")

# Crear grilla de combinaciones
grid <- expand.grid(
  cp = c(-0.5),                     # dejamos fijo como sugirió el profesor
  minbucket = c(1, 5, 10, 20),            # valores conservadores pero comunes
  maxdepth = c(4, 6, 8, 10, 12, 14),         # baja, media y alta profundidad
  minsplit = c(1000, 800, 600, 400, 200, 100, 50, 20, 10)            # uno grande y uno pequeño
)

# Inicializar tabla si no existe
if (!exists("tb_grid_search_detalle")) {
  tb_grid_search_detalle <- data.table()
}

# Total de combinaciones
total_iter <- nrow(grid)
cat("Total de combinaciones:", total_iter, "\n")

# Archivo de log
log_file <- "gridsearch_log.txt"
writeLines(paste0("🟢 Inicio grid search: ", Sys.time()), con = log_file)

# Iteración principal
for (i in seq_len(total_iter)) {
  cat("[", i, "/", total_iter, "] Ejecutando combinación...\n")
  flush.console()

  # Saltear si ya se evaluó
  if (i * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

  row <- grid[i, ]
  param_basicos <- list(
    cp        = row$cp,
    maxdepth  = row$maxdepth,
    minsplit  = row$minsplit,
    minbucket = row$minbucket
  )

  # Log inicio
  start_time <- Sys.time()

  # Ejecutar con manejo de errores
  resultado <- tryCatch({
    ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

    # Acumular resultados
    tb_grid_search_detalle <- rbindlist(
      list(tb_grid_search_detalle, rbindlist(ganancias))
    )

    # Log fin
    end_time <- Sys.time()
    tiempo <- round(difftime(end_time, start_time, units = "secs"), 2)

    log_line <- paste0(
      "[", format(Sys.time(), "%Y-%m-%d %H:%M:%S"), "] ",
      "✔ Iter ", i, "/", total_iter, " | ",
      "cp=", row$cp, ", minbucket=", row$minbucket,
      ", maxdepth=", row$maxdepth, ", minsplit=", row$minsplit,
      " | Tiempo: ", tiempo, "s"
    )
    write(log_line, file = log_file, append = TRUE)

    # Guardar cada 10 iteraciones o al final
    if (i %% 10 == 0 || i == total_iter) {
      fwrite(tb_grid_search_detalle, file = "gridsearch_detalle.txt", sep = "\t")
    }

  }, error = function(e) {
    error_line <- paste0(
      "[", format(Sys.time(), "%Y-%m-%d %H:%M:%S"), "] ",
      "❌ Error en iter ", i, "/", total_iter, " | ",
      "cp=", row$cp, ", minbucket=", row$minbucket,
      ", maxdepth=", row$maxdepth, ", minsplit=", row$minsplit,
      " | Mensaje: ", e$message
    )
    write(error_line, file = log_file, append = TRUE)
  })
}

cat("🟢 Grid search completado. Resultados guardados.\n")


[1] TRUE

[1] TRUE

Total de combinaciones: 12 
[ 1 / 12 ] Ejecutando combinación...
[ 2 / 12 ] Ejecutando combinación...
[ 3 / 12 ] Ejecutando combinación...
[ 4 / 12 ] Ejecutando combinación...
[ 5 / 12 ] Ejecutando combinación...
[ 6 / 12 ] Ejecutando combinación...
[ 7 / 12 ] Ejecutando combinación...
[ 8 / 12 ] Ejecutando combinación...
[ 9 / 12 ] Ejecutando combinación...
[ 10 / 12 ] Ejecutando combinación...
[ 11 / 12 ] Ejecutando combinación...
[ 12 / 12 ] Ejecutando combinación...
🟢 Grid search completado. Resultados guardados.


In [16]:
fwrite( tb_grid_search_detalle,
   file = "gridsearch_detalle.txt",
   sep = "\t"
)

In [17]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 12

In [18]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
912523,-0.5,6,100,5,46980000
912523,-0.5,6,100,10,47690000
912523,-0.5,10,100,5,46460000
912523,-0.5,10,100,10,46090000
912523,-0.5,14,100,5,43920000
912523,-0.5,14,100,10,42020000
912523,-0.5,6,20,5,46440000
912523,-0.5,6,20,10,47170000
912523,-0.5,10,20,5,44460000


In [19]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [20]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [21]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-0.5,6,100,10,47690000,1
-0.5,6,20,10,47170000,1
-0.5,6,100,5,46980000,1
-0.5,10,100,5,46460000,1
-0.5,6,20,5,46440000,1
-0.5,10,100,10,46090000,1
-0.5,10,20,5,44460000,1
-0.5,14,100,5,43920000,1
-0.5,10,20,10,43890000,1


In [22]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

In [26]:
library(data.table)
library(ggplot2)
library(dplyr)

# La tabla tb_grid_search ya contiene las ganancias medias
# Si necesitas cargarla desde archivo, usa esta ruta:
# ruta_archivo <- "/content/buckets/b1/exp/HT2910/gridsearch.txt"
# df <- fread(ruta_archivo)

# Usamos la tabla tb_grid_search que ya está en memoria
df <- tb_grid_search

# Ver las primeras filas
print(head(df))

# Ordenar por ganancia promedio descendente
top_ganancias <- df %>%
  arrange(desc(ganancia_mean))

# Ver las mejores combinaciones
print(head(top_ganancias, 10))

# ¿Algún valor de hiperparámetro que SIEMPRE da mal?
df %>%
  group_by(cp) %>%
  summarise(
    media_ganancia = mean(ganancia_mean, na.rm = TRUE),
    min_ganancia = min(ganancia_mean, na.rm = TRUE),
    max_ganancia = max(ganancia_mean, na.rm = TRUE)
  ) %>%
  arrange(media_ganancia) %>%
  print()

df %>%
  group_by(maxdepth) %>%
  summarise(
    media_ganancia = mean(ganancia_mean, na.rm = TRUE),
    min_ganancia = min(ganancia_mean, na.rm = TRUE),
    max_ganancia = max(ganancia_mean, na.rm = TRUE)
  ) %>%
  arrange(media_ganancia) %>%
  print()

df %>%
  group_by(minsplit) %>%
  summarise(
    media_ganancia = mean(ganancia_mean, na.rm = TRUE),
    min_ganancia = min(ganancia_mean, na.rm = TRUE),
    max_ganancia = max(ganancia_mean, na.rm = TRUE)
  ) %>%
  arrange(media_ganancia) %>%
  print()

df %>%
  group_by(minbucket) %>%
  summarise(
    media_ganancia = mean(ganancia_mean, na.rm = TRUE),
    min_ganancia = min(ganancia_mean, na.rm = TRUE),
    max_ganancia = max(ganancia_mean, na.rm = TRUE)
  ) %>%
  arrange(media_ganancia) %>%
  print()


# Combinaciones MUY malas: por debajo del percentil 10
umbral_pesimo <- quantile(df$ganancia_mean, 0.10, na.rm = TRUE)
combinaciones_pesimas <- df %>% filter(ganancia_mean <= umbral_pesimo)

# Mostrar
print("Combinaciones pésimas (por debajo del percentil 10):")
print(combinaciones_pesimas)

      cp maxdepth minsplit minbucket ganancia_mean   qty    id
   <num>    <num>    <num>     <num>         <num> <int> <int>
1:  -0.5        6      100        10      47690000     1     1
2:  -0.5        6       20        10      47170000     1     2
3:  -0.5        6      100         5      46980000     1     3
4:  -0.5       10      100         5      46460000     1     4
5:  -0.5        6       20         5      46440000     1     5
6:  -0.5       10      100        10      46090000     1     6
       cp maxdepth minsplit minbucket ganancia_mean   qty    id
    <num>    <num>    <num>     <num>         <num> <int> <int>
 1:  -0.5        6      100        10      47690000     1     1
 2:  -0.5        6       20        10      47170000     1     2
 3:  -0.5        6      100         5      46980000     1     3
 4:  -0.5       10      100         5      46460000     1     4
 5:  -0.5        6       20         5      46440000     1     5
 6:  -0.5       10      100        10      46090

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En el channel  #Labo1R-Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )